In [27]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [28]:
# Constants
G = 9.81  # Gravity (m/s^2)
BOUNCE_COEFF = 0.9  # Coefficient of restitution (energy retention on bounce)
dt = 0.01
mass = 1.0
radius = 0.2

In [29]:
# Parameters
n_balls = 100
seq_len = 160
pred_len = 40
box_size = 5.0

In [93]:
# Hyperparameters
latent_dim = 16
hidden_dim = 64
sparse_dim = 8
fourier_modes = 16
num_epochs = 1000
batch_size = 16
learning_rate = 1e-3
alpha, beta, gamma = 0.1, 0.1, 0.1

In [31]:
# Collision Handling
def handle_collisions(state, radius, box_size, bounce_coeff):
    """
    Correct position and velocity of the ball when it collides with a boundary.
    """
    x, vx, y, vy = state

    # Check and handle collisions with the left and right walls
    if x - radius < -box_size:  # Left wall
        x = -box_size + radius
        vx = -bounce_coeff * vx  # Reverse x velocity
    elif x + radius > box_size:  # Right wall
        x = box_size - radius
        vx = -bounce_coeff * vx  # Reverse x velocity

    # Check and handle collisions with the floor and ceiling
    if y - radius < -box_size:  # Floor
        y = -box_size + radius
        vy = -bounce_coeff * vy  # Reverse y velocity
    elif y + radius > box_size:  # Ceiling
        y = box_size - radius
        vy = -bounce_coeff * vy  # Reverse y velocity

    return np.array([x, vx, y, vy])

In [32]:
# Particle Dynamics
def particle_dynamics_with_collisions(t, y, mass, radius, box_size, drag_coeff, bounce_coeff):
    """
    Simulate the dynamics of a ball with gravity, air resistance, and proper collision handling.
    """
    # Unpack state variables
    x, vx, y_pos, vy = y
    dydt = np.zeros_like(y)

    # Update positions
    dydt[0] = vx  # dx/dt = vx
    dydt[2] = vy  # dy/dt = vy

    # Forces: Gravity and Air Resistance
    speed = np.sqrt(vx**2 + vy**2)  # Compute speed for drag calculation
    drag_force_x = -drag_coeff * speed * vx  # Drag force in x-direction
    drag_force_y = -drag_coeff * speed * vy  # Drag force in y-direction

    dydt[1] = drag_force_x / mass  # dvx/dt = Drag force in x / mass
    dydt[3] = (-G + drag_force_y / mass)  # dvy/dt = Gravity + drag force in y / mass

    return dydt

In [33]:
# Generate GIFs
def generate_ball_gif(trajectory, radius, box_size, save_path):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-box_size, box_size)
    ax.set_ylim(-box_size, box_size)
    ax.set_aspect('equal')

    ball = plt.Circle((trajectory[0, 0], trajectory[0, 2]), radius, fc='blue')
    ax.add_patch(ball)

    def update(frame):
        ball.set_center((trajectory[frame, 0], trajectory[frame, 2]))
        return ball,

    ani = FuncAnimation(fig, update, frames=len(trajectory), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [34]:
# Dataset Generation
def generate_ball_dataset(n_balls, dt, seq_len, pred_len, box_size=5.0, output_dir="Ball Dataset"):
    os.makedirs(output_dir, exist_ok=True)
    trajectories = []
    for i in tqdm(range(n_balls), desc="Generating dataset"):
        mass = np.random.uniform(0.5, 5.0)
        radius = np.random.uniform(0.05, 0.2)
        drag_coeff = 0.1
        bounce_coeff = 0.9
        init_pos = np.random.uniform(-box_size + radius, box_size - radius, 2)
        init_vel = np.random.uniform(-5.0, 5.0, 2)
        y0 = np.concatenate([init_pos, init_vel])
        t_eval = np.linspace(0, (seq_len + pred_len) * dt, int(seq_len + pred_len))
        trajectory = []
        state = y0
        for t in t_eval:
            sol = solve_ivp(
                particle_dynamics_with_collisions,
                (t, t + dt),
                state,
                args=(mass, radius, box_size, drag_coeff, bounce_coeff),
                method="RK45",
                t_eval=[t + dt]
            )
            if sol.y.size == 0:
                break
            state = sol.y.flatten()
            state = handle_collisions(state, radius, box_size, bounce_coeff)
            trajectory.append(state)

        trajectories.append(np.array(trajectory))
        gif_path = os.path.join(output_dir, f"ball_{i}.gif")
        generate_ball_gif(np.array(trajectory), radius, box_size, gif_path)
    return trajectories

In [35]:
# Generate dataset
trajectories = generate_ball_dataset(n_balls, dt, seq_len + pred_len, box_size)

Generating dataset: 100%|██████████| 100/100 [13:09<00:00,  7.89s/it]


In [36]:
# Dataset Class
class BallTrajectoryDataset(Dataset):
    def __init__(self, trajectories, seq_len, pred_len):
        self.trajectories = trajectories
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        trajectory = self.trajectories[idx]
        input_seq = trajectory[:self.seq_len]
        target_seq = trajectory[self.seq_len:self.seq_len + self.pred_len]
        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32)

In [37]:
def save_predictions(predictions, targets, folder, radius, box_size, mode="test"):
    os.makedirs(folder, exist_ok=True)
    real_folder = os.path.join(folder, "real_movement")
    predicted_folder = os.path.join(folder, "predicted_movement")
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(predicted_folder, exist_ok=True)

    for idx in range(len(predictions)):
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"{mode}_real_{idx}.gif"), "True Trajectory")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"{mode}_predicted_{idx}.gif"), "Predicted Trajectory")

In [38]:
# Save Trajectory GIFs
def save_trajectory_gif(trajectory, radius, box_size, save_path, title="Trajectory"):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-box_size, box_size)
    ax.set_ylim(-box_size, box_size)
    ax.set_aspect('equal')
    ax.set_title(title)

    ball = plt.Circle((trajectory[0, 0], trajectory[0, 2]), radius, fc='blue')
    ax.add_patch(ball)

    def update(frame):
        ball.set_center((trajectory[frame, 0], trajectory[frame, 2]))
        return ball,

    ani = FuncAnimation(fig, update, frames=len(trajectory), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [39]:
# Save Best and Worst Predictions
def save_best_worst_predictions(predictions, targets, losses, folder, radius, box_size, mode):
    os.makedirs(folder, exist_ok=True)
    real_folder = os.path.join(folder, "real_movement")
    predicted_folder = os.path.join(folder, "predicted_movement")
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(predicted_folder, exist_ok=True)

    sorted_indices = np.argsort(losses)
    best_indices = sorted_indices[:5]
    worst_indices = sorted_indices[-5:]

    for idx in best_indices:
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"{mode}_best_real_{idx}.gif"), "True Trajectory (Best)")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"{mode}_best_predicted_{idx}.gif"), "Predicted Trajectory (Best)")

    for idx in worst_indices:
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"{mode}_worst_real_{idx}.gif"), "True Trajectory (Worst)")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"{mode}_worst_predicted_{idx}.gif"), "Predicted Trajectory (Worst)")

In [40]:
# Save All Test Predictions
def save_all_test_predictions(predictions, targets, folder, radius, box_size):
    os.makedirs(folder, exist_ok=True)
    real_folder = os.path.join(folder, "real_movement")
    predicted_folder = os.path.join(folder, "predicted_movement")
    os.makedirs(real_folder, exist_ok=True)
    os.makedirs(predicted_folder, exist_ok=True)

    for idx in range(len(predictions)):
        save_trajectory_gif(targets[idx][-len(targets[idx]) // 5:], radius, box_size, os.path.join(real_folder, f"test_real_{idx}.gif"), "True Trajectory")
        save_trajectory_gif(predictions[idx][-len(predictions[idx]) // 5:], radius, box_size, os.path.join(predicted_folder, f"test_predicted_{idx}.gif"), "Predicted Trajectory")

In [86]:
# Hybrid Model
class SINDyKoopmanFNO(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim, sparse_dim, fourier_modes, pred_len):
        super(SINDyKoopmanFNO, self).__init__()
        self.pred_len = pred_len
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
        )
        self.koopman = nn.Linear(latent_dim, latent_dim, bias=False)
        self.sindy_dynamics = nn.Linear(latent_dim, sparse_dim, bias=False)
        self.fourier_layer = nn.Sequential(
            nn.Linear(latent_dim, fourier_modes),
            nn.ReLU(),
            nn.Linear(fourier_modes, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def forward(self, x):
        z = self.encoder(x)  # z: [batch_size, seq_len, latent_dim]
        predictions = []
        z_current = z[:, 0, :]
        z_next_list = []
        dzdt_list = []

        for _ in range(self.pred_len):
            z_next = self.koopman(z_current)  # Koopman dynamics
            dzdt = self.sindy_dynamics(z_next)  # Sparse dynamics
            z_fourier = self.fourier_layer(z_next)  # Fourier transform
            z_combined = z_next + z_fourier  # Combine latent dynamics
            x_reconstructed = self.decoder(z_combined)  # Decode back to input
            predictions.append(x_reconstructed.unsqueeze(1))
            z_next_list.append(z_next.unsqueeze(1))
            dzdt_list.append(dzdt.unsqueeze(1))
            z_current = z_next

        predictions = torch.cat(predictions, dim=1)  # [batch_size, pred_len, input_dim]
        z_next_all = torch.cat(z_next_list, dim=1)  # [batch_size, pred_len, latent_dim]
        dzdt_all = torch.cat(dzdt_list, dim=1)  # [batch_size, pred_len, sparse_dim]
        return predictions, z_next_all, dzdt_all, z[:, :self.pred_len, :]


In [87]:
# Loss Functions
def compute_energy(state, mass, g):
    vx, vy = state[:, :, 1], state[:, :, 3]
    y = state[:, :, 2]
    kinetic_energy = 0.5 * mass * (vx**2 + vy**2)
    potential_energy = mass * g * y
    return kinetic_energy + potential_energy

def energy_loss(predicted_state, true_state, mass, g):
    predicted_energy = compute_energy(predicted_state, mass, g)
    true_energy = compute_energy(true_state, mass, g)
    return torch.mean((predicted_energy - true_energy)**2)

def combined_loss(x_reconstructed, x_target, z_next, dzdt, z_pred, mass, g, alpha=0.1, beta=0.1, gamma=0.1):
    reconstruction_loss = torch.mean((x_reconstructed - x_target) ** 2)
    koopman_loss = torch.mean((z_next - z_pred) ** 2)
    
    # Map z_pred to sparse_dim for comparison with dzdt
    z_pred_sparse = dzdt.new_zeros(dzdt.shape)  # Match shape
    for i in range(dzdt.size(1)):  # Loop over sequence length
        z_pred_sparse[:, i, :] = dzdt[:, i, :]  # Direct mapping for simplicity
    
    sparse_loss = torch.mean((dzdt - z_pred_sparse) ** 2)
    energy_loss_value = energy_loss(x_reconstructed, x_target, mass, g)
    return reconstruction_loss + alpha * energy_loss_value + beta * koopman_loss + gamma * sparse_loss

In [88]:
# Dataset Splitting
def split_dataset(trajectories, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    total = train_ratio + val_ratio + test_ratio
    train_ratio /= total
    val_ratio /= total
    test_ratio /= total

    train_val_data, test_data = train_test_split(trajectories, test_size=test_ratio, random_state=42)
    train_data, val_data = train_test_split(train_val_data, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)
    return train_data, val_data, test_data

In [89]:
train_set, val_set, test_set = split_dataset(trajectories, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
train_dataset = BallTrajectoryDataset(train_set, seq_len, pred_len)
val_dataset = BallTrajectoryDataset(val_set, seq_len, pred_len)
test_dataset = BallTrajectoryDataset(test_set, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [90]:
# Training and Validation
def train_and_save_predictions(model, train_loader, val_loader, optimizer, mass, g, num_epochs, alpha, beta, gamma, radius, box_size):
    model.train()
    for epoch in range(num_epochs):
        total_train_loss, total_train_acc = 0.0, 0.0
        total_val_loss, total_val_acc = 0.0, 0.0
        train_predictions, train_targets, train_losses = [], [], []

        # Training loop
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            x_reconstructed, z_next, dzdt, z = model(inputs)
            z_pred = model.koopman(z)
            loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z_pred, mass, g, alpha, beta, gamma)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            train_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
            total_train_acc += train_acc.item()

            train_predictions.append(x_reconstructed.detach().cpu().numpy())
            train_targets.append(targets.detach().cpu().numpy())
            train_losses.append(loss.item())

        val_predictions, val_targets, val_losses = [], [], []
        model.eval()
        with torch.no_grad():
            for inputs, targets in val_loader:
                x_reconstructed, z_next, dzdt, z = model(inputs)
                z_pred = model.koopman(z)
                loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z_pred, mass, g, alpha, beta, gamma)
                total_val_loss += loss.item()
                val_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
                total_val_acc += val_acc.item()

                val_predictions.append(x_reconstructed.detach().cpu().numpy())
                val_targets.append(targets.detach().cpu().numpy())
                val_losses.append(loss.item())

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {total_train_loss / len(train_loader):.4f}, Train Accuracy: {total_train_acc / len(train_loader):.4f}, "
              f"Val Loss: {total_val_loss / len(val_loader):.4f}, Val Accuracy: {total_val_acc / len(val_loader):.4f}")

        if epoch == num_epochs - 1:
            save_best_worst_predictions(np.concatenate(train_predictions), np.concatenate(train_targets), train_losses, "training_predictions", radius, box_size, "train")
            save_best_worst_predictions(np.concatenate(val_predictions), np.concatenate(val_targets), val_losses, "val_predictions", radius, box_size, "val")

In [91]:
# Initialize model and optimizer
model = SINDyKoopmanFNO(input_dim=4, latent_dim=latent_dim, hidden_dim=hidden_dim, sparse_dim=sparse_dim, fourier_modes=fourier_modes, pred_len=pred_len)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [94]:
# Train the model
train_and_save_predictions(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    mass=mass,
    g=G,
    num_epochs=num_epochs,
    alpha=alpha,
    beta=beta,
    gamma=gamma,
    radius=radius,
    box_size=box_size
)

Epoch 1/1000, Train Loss: 7.3872, Train Accuracy: 0.2195, Val Loss: 10.6601, Val Accuracy: 0.2075
Epoch 2/1000, Train Loss: 6.9944, Train Accuracy: 0.2062, Val Loss: 11.0481, Val Accuracy: 0.2119
Epoch 3/1000, Train Loss: 7.5607, Train Accuracy: 0.1995, Val Loss: 10.5526, Val Accuracy: 0.2106
Epoch 4/1000, Train Loss: 9.1961, Train Accuracy: 0.1916, Val Loss: 11.1055, Val Accuracy: 0.2152
Epoch 5/1000, Train Loss: 8.9021, Train Accuracy: 0.1906, Val Loss: 10.5433, Val Accuracy: 0.2108
Epoch 6/1000, Train Loss: 7.2979, Train Accuracy: 0.2033, Val Loss: 10.2374, Val Accuracy: 0.2133
Epoch 7/1000, Train Loss: 7.5197, Train Accuracy: 0.2100, Val Loss: 10.3335, Val Accuracy: 0.2140
Epoch 8/1000, Train Loss: 7.5334, Train Accuracy: 0.2009, Val Loss: 10.2805, Val Accuracy: 0.2139
Epoch 9/1000, Train Loss: 7.4942, Train Accuracy: 0.2025, Val Loss: 10.6869, Val Accuracy: 0.2166
Epoch 10/1000, Train Loss: 8.0718, Train Accuracy: 0.1947, Val Loss: 10.4337, Val Accuracy: 0.2138
Epoch 11/1000, Trai

KeyboardInterrupt: 

In [ ]:
# Test Evaluation
def evaluate_and_save_test_predictions(model, test_loader, mass, g, radius, box_size):
    model.eval()
    test_predictions, test_targets = [], []
    total_test_loss, total_test_acc = 0.0, 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            x_reconstructed, z_next, dzdt, z = model(inputs)
            loss = combined_loss(x_reconstructed, targets, z_next, dzdt, z, mass, g)
            total_test_loss += loss.item()
            test_acc = 1 - (torch.norm(x_reconstructed - targets) / torch.norm(targets))
            total_test_acc += test_acc.item()

            test_predictions.append(x_reconstructed.detach().cpu().numpy())
            test_targets.append(targets.detach().cpu().numpy())

    avg_test_loss = total_test_loss / len(test_loader)
    avg_test_acc = total_test_acc / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_acc:.4f}")

    save_all_test_predictions(np.concatenate(test_predictions), np.concatenate(test_targets), folder="test_predictions", radius=radius, box_size=box_size)

In [ ]:
# Evaluate the model
evaluate_and_save_test_predictions(model, test_loader, mass, G, radius, box_size)